# Se pretende predecir a través de un dataset de kaggle que nota de matemáticas obtendrá un estudiante a partir de su nota de lengua y escritura y variables sociales.
- Detecto que no todas las variables sociales influyen por lo que se eliminan datos como la influencia de los estudios de los padres a excepción de una clase que si afecta y lo he mismo en caso de la étnia.
- Se escala el modelo al tener diferencias grandes de rango entre las notas y las variables sociales. Finalmente se comprueba que los resultados del modelo son idénticos sin el escalado de las mismas.
- Se estudian modelos en árbol y modelos de regresión lineal. En este caso hay una gran linealidad en los resultados por lo que se obtienen mejores resultados con estos modelos y se utiliza el modelo más simple, porque los resultados son practicamente idénticos al resto de modelos lineales más complejos. Linear Regression.

In [ ]:
# === Bloque 1: Prepara X_test e y_test exactamente como en tu entrenamiento ===
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

url = "https://raw.githubusercontent.com/Aiadevop/math_student_grade/refs/heads/main/data/raw/StudentsPerformance.csv"
df_rg = pd.read_csv(url)

# 1) Limpiar nombres de columnas
df_rg.columns = (
    df_rg.columns
    .str.strip()
    .str.replace(" ", "_")
    .str.replace("'", "", regex=False)
    .str.replace("/", "_", regex=False)
)

# 2) Limpiar strings en celdas
df_rg = df_rg.apply(lambda col: col.map(lambda x: x.strip().replace("'", "").replace(" ", "_") if isinstance(x, str) else x))

# 3) Codificación idéntica a tu pipeline
df_num = df_rg.copy()

label_encoders = {
    'gender': {'female': 0, 'male': 1},
    'lunch': {'free/reduced': 0, 'standard': 1},
    'test_preparation_course': {'none': 0, 'completed': 1}
}
for col, mapping in label_encoders.items():
    df_num[col] = df_num[col].map(mapping)

ohe_cols = ['race_ethnicity', 'parental_level_of_education']
try:
    ohe = OneHotEncoder(drop='first', sparse_output=False)
except TypeError:
    ohe = OneHotEncoder(drop='first', sparse=False)

ohe_result = ohe.fit_transform(df_num[ohe_cols])
ohe_df = pd.DataFrame(ohe_result, columns=ohe.get_feature_names_out(ohe_cols), index=df_num.index)

df_num = df_num.drop(columns=ohe_cols)
df_num = pd.concat([df_num, ohe_df], axis=1)

# 4) Definir X e y (misma selección de columnas que usaste)
drop_cols = [
    'race_ethnicity_group_B','race_ethnicity_group_C','race_ethnicity_group_D',
    'parental_level_of_education_bachelors_degree',
    'parental_level_of_education_masters_degree',
    'parental_level_of_education_some_college',
    'parental_level_of_education_some_high_school'
]
df_lr = df_num.drop(columns=drop_cols)
X = df_lr.drop(columns=['math_score'])
y = df_lr['math_score']

# 5) Split idéntico
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Listo: X_test e y_test preparados.")
print("Tamaño X_test:", X_test.shape)


Listo: X_test e y_test preparados.
Tamaño X_test: (200, 7)


In [ ]:
df_lr.head()

,gender,lunch,test_preparation_course,math_score,reading_score,writing_score,race_ethnicity_group_E,parental_level_of_education_high_school
0,0,1,0,72,72,74,0.0,0.0
1,0,1,1,69,90,88,0.0,0.0
2,0,1,0,90,95,93,0.0,0.0
3,1,0,0,47,57,44,0.0,0.0
4,1,1,0,76,78,75,0.0,0.0


In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

# Diccionario con los modelos a probar
modelos = {
    "LinearRegression": LinearRegression(),
    # "Ridge": Ridge(alpha=1.0),
    # "Lasso": Lasso(alpha=0.01),
    # "ElasticNet": ElasticNet(alpha=0.01, l1_ratio=0.5)
}

# DataFrame para guardar resultados
resultados = []

# Entrenar y evaluar cada modelo
for nombre, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    resultados.append({
        "Modelo": nombre,
        "MAE": mae,
        "RMSE": rmse,
        "R²": r2
    })

    if nombre == "LinearRegression":
      lin_reg = modelo

# Convertir a DataFrame para comparar
df_resultados = pd.DataFrame(resultados).sort_values(by="R²", ascending=False)

print("📊 Resultados de los modelos lineales:")
display(df_resultados)

📊 Resultados de los modelos lineales:


,Modelo,MAE,RMSE,R²
0,LinearRegression,4.17494,5.348903,0.882424


In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd

# Definimos los modelos
modelos = {
    "LinearRegression": LinearRegression()
}

# Definimos la validación cruzada
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Scorers personalizados
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
rmse_scorer = make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)),
                          greater_is_better=False)

# Guardar resultados
resultados = []

for nombre, modelo in modelos.items():
    r2_scores = cross_val_score(modelo, X, y, cv=cv, scoring="r2")
    mae_scores = cross_val_score(modelo, X, y, cv=cv, scoring=mae_scorer)
    rmse_scores = cross_val_score(modelo, X, y, cv=cv, scoring=rmse_scorer)

    resultados.append({
        "Modelo": nombre,
        "R² (mean)": np.mean(r2_scores),
        "R² (std)": np.std(r2_scores),
        "MAE (mean)": -np.mean(mae_scores),
        "RMSE (mean)": -np.mean(rmse_scores)
    })

# Convertir a DataFrame
df_cv = pd.DataFrame(resultados).sort_values(by="R² (mean)", ascending=False)

print("📊 Resultados con validación cruzada (5 folds):")
display(df_cv)

📊 Resultados con validación cruzada (5 folds):


,Modelo,R² (mean),R² (std),MAE (mean),RMSE (mean)
0,LinearRegression,0.872151,0.012842,4.314648,5.398176


In [ ]:
# Tras fit:
# lin_reg ya está entrenado en tu bloque previo
modelo = lin_reg  # <- modelo en variable

# Registro nuevo (sin escalar)
nuevo = {
    "reading_score": 78,
    "writing_score": 84,
    "lunch": 0,
    "race_ethnicity_group_E": 1,
    "test_preparation_course": 1,
    "gender": 1,
    "parental_level_of_education_high_school": 1
}

import pandas as pd

# Orden y columnas EXACTAS como en X
cols = [
    "gender",
    "lunch",
    "test_preparation_course",
    "reading_score",
    "writing_score",
    "race_ethnicity_group_E",
    "parental_level_of_education_high_school"
]

X_new = pd.DataFrame([nuevo], columns=cols)

# (Opcional, a prueba de balas) Alinear con las columnas reales de X
# X_new = X_new.reindex(columns=X.columns, fill_value=0)

pred = modelo.predict(X_new)[0]
print("Predicción math_score:", pred)


Predicción math_score: 86.71655878604037


In [ ]:
import joblib
from google.colab import files

# Guardar el mejor modelo como archivo .pkl
joblib.dump(lin_reg, 'lin_reg_model_opt.pkl')

# Descargar el modelo guardado a tu PC
files.download('lin_reg_model_opt.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# === Bloque 2: Carga el .pkl y valida sobre X_test ===
import joblib
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Cargar el modelo guardado
modelo = joblib.load('lin_reg_model_opt.pkl')

# Predicción y métricas
y_pred = modelo.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("MAE:", mae)
print("RMSE:", rmse)
print("R2:", r2)

# Muestra mínima para inspección manual
print("Primeras_5_predicciones:", np.round(y_pred[:5], 3))
print("Primeros_5_valores_reales:", y_test.iloc[:5].to_list())


MAE: 4.174939680743238
RMSE: 5.348902897486917
R2: 0.8824239733361281
Primeras_5_predicciones: [76.823 57.216 78.135 76.215 86.943]
Primeros_5_valores_reales: [91, 53, 80, 74, 84]


In [ ]:
nuevo = {
    "gender": 1,
    "lunch": 0,
    "test_preparation_course": 1,
    "reading_score": 78,
    "writing_score": 84,
    "race_ethnicity_group_E": 1,
    "parental_level_of_education_high_school": 1
}


In [ ]:
import numpy as np
import pandas as pd
import joblib

# Definir orden exacto de columnas como en entrenamiento
cols = [
    "gender",
    "lunch",
    "test_preparation_course",
    "reading_score",
    "writing_score",
    "race_ethnicity_group_E",
    "parental_level_of_education_high_school"
]

# Registro en DataFrame
nuevo_df = pd.DataFrame([nuevo], columns=cols)

# Parámetros del scaler
mean = np.array([0.482, 0.645, 0.358, 69.169, 68.054, 0.14, 0.196])
scale = np.array([0.4996759, 0.4785133, 0.4794121, 14.5928900, 15.1880573, 0.3469870, 0.3969685])

# Escalar manualmente
nuevo_scaled = (nuevo_df.values - mean) / scale


In [ ]:
# Cargar modelo previamente guardado
modelo = joblib.load("lin_reg_model_opt.pkl")

# Predicción
y_pred = modelo.predict(nuevo_scaled)

print("Predicción math_score:", y_pred[0])


Predicción math_score: 12.265761381443799


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
